In [1]:
import numpy as np
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
import time
import re

In [2]:
book_titles = list()
book_reviews = list() 
reviews_date = list()

In [3]:
data = pd.read_csv('yes24_자기개발_2008_2021.csv', encoding='cp949')
data = data[1:]
data['Column5'] = data['Column5'].astype('int')
book_list = list(data[data['Column5'] >= 2018]['Column3']) and list(data[data['Column5'] <= 2021]['Column3'])[::-1]
book_list

['맥스웰 몰츠 성공의 법칙',
 '경험의 함정',
 '요즘 것들의 사생활 : 먹고사니즘',
 '어떻게 인생을 살 것인가 (특별판 양장)',
 '리얼리티 트랜서핑 3',
 '도파민형 인간',
 '루이스 헤이의 치유 수업',
 '그렇게 심각할 필요 없어',
 '행복의 선택',
 '폰더 씨의 위대한 하루',
 '이왕 시작한 거 딱, 100일만 달려 볼게요',
 '페어 플레이 프로젝트',
 '더 룰 (THE RULE) - 리치 편',
 '승리, 패배, 그리고 교훈',
 'THE MAGIC 매직',
 '당신만을 위한 100개의 질문',
 '운의 알고리즘',
 '아마존처럼 회의하라',
 '마음챙김이 일상이 되면 달라지는 것들',
 '조인트 사고',
 '핵심만 남기고 줄이는 게 체질',
 '못난 게 아니라, 조금 서툰 겁니다',
 '터지는 콘텐츠는 이렇게 만듭니다',
 '오늘부터 그 자리에 의자를 두기로 했다',
 '1회성 고객을 100번 방문 고객으로 만드는 비밀',
 '서동주의 합격 공부법',
 '나는 저녁마다 삶의 방향을 잡는다',
 '생각하라 그리고 부자가 되어라',
 '나폴레온 힐 부자 수업',
 '클린 CLEAN',
 '팀장의 말투',
 '오늘부터 나는 브랜드가 되기로 했다',
 '당신은 다른 사람의 성공에 기여한 적 있는가?',
 '나는 아침마다 삶의 감각을 깨운다',
 '리더의 말 그릇',
 '나는 이제 마음 편히 살기로 했다',
 '생각의 쓰임',
 '프리워커스',
 '인간을 바꾸는 5가지 법칙',
 '눈치껏 못 배웁니다, 일센스',
 '똑똑하게 생존하기',
 '무조건 팔리는 카피 단어장',
 '일 잘하는 사람은 글을 잘 씁니다',
 '5초의 법칙',
 '파워블로거 핑크팬더의 블로그 글쓰기',
 '종이 위의 기적 쓰면 이루어진다',
 '나는 디지털노마드맘으로 살기로 했다',
 '나는 자는 동안에도 돈을 번다',
 '짧은 말이 무기가 된다',
 '3개의 소원 100일의 기적',
 '심리학이 돈을 말하다',
 '프레젠테이션 코칭 북 PresenTation C

In [5]:
url='http://www.yes24.com/24/Category/Display/001001026'
driver = webdriver.Chrome(executable_path=r'C:\MIN\1. Python\chromedriver_win32\chromedriver.exe')
driver.get(url)
driver.implicitly_wait(3)
just_book_list = list()
finish_book_list = list()
num=int(0)
list_num = int(0)

In [11]:
num = 26

In [12]:
while True:
#     driver.switch_to.window(window_name=tab)
    num+=int(1)
    if len(book_list[(num-1)*7:num*7])==0:
        print('process finished')
        break        
    try:
        just_book_list = book_list[(num-1)*7:num*7]
#         just_book_list = ['사람을 얻는 기술']
        print(just_book_list)
    except IndexError:
        just_book_list = book_list[(num-1)*7:]
    for i in just_book_list:
        tab = driver.window_handles[0]
        driver.switch_to.window(window_name=tab)
        search = driver.find_element_by_css_selector('#query')
        search.clear() #검색창 입력어 지우기
        search.send_keys(i) #새로운 검색어 입력|
        click = driver.find_element_by_css_selector('#yesSForm > fieldset > span.schBtn > button')
        click.send_keys(Keys.CONTROL +"\n") #클릭해서 새 탭으로 열리기
    while True:
        try:
            tab = driver.window_handles[1]
            driver.implicitly_wait(3)
            driver.switch_to.window(window_name=tab)
            try:
                driver.find_element_by_css_selector('#divYes24SCMEvent > div.yPopBot > div.pTxt > label').click()
#                 time.sleep(2) #테스트
            except ElementNotInteractableException:
                pass
            except ElementNotInteractableException:
                driver.close()
                pass
#             try:
            title = driver.find_element_by_xpath(f'/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]/p[1]/a').text
            if title in just_book_list:
                u=1
            if title not in just_book_list:
                driver.find_element_by_css_selector('#formTest > ul > li:nth-child(2) > a').send_keys(Keys.ENTER)
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]/p[1]/a').text
                u=1
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[3]/td[2]/p[1]/a').text
                u=3
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[5]/td[2]/p[1]/a').text
                u=5
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[7]/td[2]/p[1]/a').text
                u=7
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[9]/td[2]/p[1]/a').text
                u=9
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[11]/td[2]/p[1]/a').text
                u=11
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[13]/td[2]/p[1]/a').text
                u=13
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[15]/td[2]/p[1]/a').text
                u=15
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[17]/td[2]/p[1]/a').text
                u=17
            if title in just_book_list:
                driver.find_element_by_xpath(f'/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[{u}]/td[2]/p[1]/a').click()


#             except ElementNotInteractableException:
#                 driver.close()
            time.sleep(2)
            list_num +=1
            try:
#                 yes = driver.find_element_by_css_selector('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_ratingArea > span.gd_reviewCount > a')
#                 yes.click() #구매리뷰
                title = driver.find_element_by_xpath('/html/body/div/div[4]/div[2]/div[1]/div/h2')
                title = title.text
                
                if title in just_book_list:
                    print(f'[{list_num}] {title}')
                    finish_book_list.append(title)
                    for k in list(range(2)):
                        for j in list(range(40)):    
                            for i in list(range(5)):
#                                 yes = driver.find_element_by_css_selector('#infoset_reviewContentList > div:nth-child('+ str(i+3) + ') > div.reviewInfoBot.crop > a > div > span')
                                element = driver.find_element_by_xpath('//*[@id="infoset_reviewContentList"]/div['+str(i+2)+']/div[2]/a/div/span')
                                driver.execute_script("arguments[0].click();", element)
                                time.sleep(0.5) #에러날 경우 시간 늘리기
                            for i in list(range(5)):
    #                             print(i+1)
                                yes = driver.find_element_by_xpath('//*[@id="infoset_reviewContentList"]/div['+ str(i+2) +']/div[3]/div[2]')
                                date = driver.find_element_by_xpath('//*[@id="infoset_reviewContentList"]/div[2]/div[1]/div/em[4]')
    #                             print(date.text)
    #                             print(yes.text)
    #                             print()
                                book_titles.append(title)
                                book_reviews.append(yes.text) 
                                reviews_date.append(date.text)
    #                         print('-------------------------------------페이지 '+ str(k) + str(j+1) + ' 끝---------------------------------------')
                            yes = driver.find_element_by_css_selector('#infoset_reviewContentList > div.review_sort.sortBot > div.review_sortLft > div > a:nth-child(' + str(j+4) +')')
                            yes.click()
    #                         print()
                            time.sleep(1)
#                 else:
#                     driver.back()
#                     driver.implicitly_wait(3)
#                     driver.close()
#                     print(f'[{list_num}] {title}: 검색한 책이름과 페이지에서 찾은 책 이름이 달라 패스합니다')

                        
            except NoSuchElementException:                
                driver.close()
                pass
            except ElementNotInteractableException:
                driver.close()
                pass
            except ElementClickInterceptedException :
                driver.close()
                pass
        except IndexError:
            print('start new books process')
            break
#         break
    
#         

['인생도 복리가 됩니다', '어른의 말공부', '1페이지 꿈지도', '실행이 답이다', '도티의 플랜B', '불렛저널', '압축 공부']
[182] 인생도 복리가 됩니다
[183] 어른의 말공부
[184] 1페이지 꿈지도
[185] 실행이 답이다
[186] 도티의 플랜B
[187] 불렛저널
[188] 압축 공부
start new books process
['타인을 읽는 말', '일을 잘한다는 것', '단단한 여자', '그렇게 말해줘서 고마워', '아침이 달라지는 저녁 루틴의 힘', '객관적이고 과학적인 공부법', '마음챙김']
[189] 타인을 읽는 말
[190] 일을 잘한다는 것
[191] 단단한 여자
[192] 그렇게 말해줘서 고마워
[193] 아침이 달라지는 저녁 루틴의 힘
[194] 객관적이고 과학적인 공부법
[195] 마음챙김
start new books process
['감정의 발견', '몰입 Think hard!', '말투 하나로 의외로 잘 되기 시작했다', '돈보다 운을 벌어라', '50부터는 물건은 뺄셈 마음은 덧셈', '치유', '빨강 머리 앤이 3년 후 나에게 : Q & A a day (램스킨 리미티드 에디션)']
[196] 감정의 발견
[197] 몰입 Think hard!
[198] 말투 하나로 의외로 잘 되기 시작했다
[199] 돈보다 운을 벌어라
[200] 50부터는 물건은 뺄셈 마음은 덧셈
[202] 빨강 머리 앤이 3년 후 나에게 : Q & A a day (램스킨 리미티드 에디션)
start new books process
['한 장 보고서의 정석', '신의 직장은 아니지만 공기업은 가고 싶어', '심리학이 이렇게 쓸모 있을 줄이야', '기획의 정석', '마케터의 문장', '내 인생을 바꾼 한 권의 책', '나는 왜 도와달라는 말을 못할까']
[203] 한 장 보고서의 정석
[204] 신의 직장은 아니지만 공기업은 가고 싶어
[205] 심리학이 이렇게 쓸모 있을 줄이야
[206] 기획의 정석
[207] 마케터의 문장
[208] 내

NameError: name 'ElementClickInterceptedException' is not defined

밑 코드 만들긴 했는데 테스트 안해봤슴당 잘 돌아가는지 확인해봐주세엽

In [7]:
reviews = pd.DataFrame(book_reviews)
titles = pd.DataFrame(book_titles)
dates = pd.DataFrame(reviews_date)
final_data= pd.concat([titles, reviews, dates],axis=1)
final_data

,0,0,0
0,맥스웰 몰츠 성공의 법칙,자아 이미지\n 흠......\n 자기계발 관련 고전? 알고 보니 꽤 유명한 책...,2021-05-06
1,맥스웰 몰츠 성공의 법칙,제목 그대로 성공의 법칙에 관해 말해주는 책.\n도서관에서 빌려서 읽다가 종이책으로...,2021-05-06
2,맥스웰 몰츠 성공의 법칙,"다양한 자기계발서를 접했고, 성공에 대한 방법론적인 책을 많이 보았다.\n그런데 맥...",2021-05-06
3,맥스웰 몰츠 성공의 법칙,맥스웰몰츠 성공의 법칙\n: 부와 성공을 부르는 마음의 법칙 사이코사이버네틱스\n\...,2021-05-06
4,맥스웰 몰츠 성공의 법칙,-. 나에게 어떤 점이 유용한가?\n : 처음 이 책을 접했을 때 흔하디 흔한...,2021-05-06
...,...,...,...
4418,하루 한 장 마음챙김,코로나19로 인해 계획했던 일들이 조금씩 어그러지며 지지부진했던 한 해를 보내고...,2021-01-30
4419,하루 한 장 마음챙김,루이즈 헤이의 하루 한 장 마음 챙김.\n루이즈 헤이 여사가 돌아가시기 전에 기획했...,2021-01-30
4420,하루 한 장 마음챙김,"루이스 헤이 저의 『하루 한 장 마음 챙김』을 읽고\n좋은 글귀 하나가, 마음 챙김...",2021-01-16
4421,하루 한 장 마음챙김,만병의 근원은 마음에서 비롯된다는 말이 있다.\n다시 말하면 스트레스나 마음의 환상...,2021-01-16


In [9]:
final_data.to_csv('2018년-2021년 자기개발 베스트셀러 리뷰.csv', encoding='utf-8')